In [3]:
import tensorflow as tf
import horovod.tensorflow.keras as hvd

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

In [4]:
(mnist_images, mnist_labels), _ = \
    tf.keras.datasets.mnist.load_data(path='mnist-%d.npz' % hvd.rank())

dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
             tf.cast(mnist_labels, tf.int64))
)
dataset = dataset.repeat().shuffle(10000).batch(128)

11501568/11490434 [==============================] - 2s 0us/step


2022-04-16 13:06:57.597546: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-16 13:07:00.961781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30953 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


In [6]:
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
# Horovod: adjust learning rate based on number of GPUs.
scaled_lr = 0.001 * hvd.size()
opt = tf.optimizers.Adam(scaled_lr)

# Horovod: add Horovod DistributedOptimizer.
opt = hvd.DistributedOptimizer(
    opt, backward_passes_per_step=1, average_aggregated_gradients=True)

# Horovod: Specify `experimental_run_tf_function=False` to ensure TensorFlow
# uses hvd.DistributedOptimizer() to compute gradients.
mnist_model.compile(loss=tf.losses.SparseCategoricalCrossentropy(),
                    optimizer=opt,
                    metrics=['accuracy'],
                    experimental_run_tf_function=False)

In [9]:
callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    #
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first three epochs. See https://arxiv.org/abs/1706.02677 for details.
    hvd.callbacks.LearningRateWarmupCallback(initial_lr=scaled_lr, warmup_epochs=3, verbose=1),
]

In [10]:
# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(tf.keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))
# Horovod: write logs on worker 0.
verbose = 1 if hvd.rank() == 0 else 0

# Train the model.
# Horovod: adjust number of steps based on number of GPUs.
mnist_model.fit(dataset, steps_per_epoch=500 // hvd.size(), callbacks=callbacks, epochs=24, verbose=verbose)

Epoch 1/24


2022-04-16 13:40:45.135369: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-04-16 13:40:48.562838: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


500/500 [==============================] - 21s 12ms/step - loss: 0.2162 - accuracy: 0.9344 - lr: 0.0010
Epoch 2/24
500/500 [==============================] - 6s 12ms/step - loss: 0.0785 - accuracy: 0.9761 - lr: 0.0010
Epoch 3/24
499/500 [============================>.] - ETA: 0s - loss: 0.0583 - accuracy: 0.9824
Epoch 3: finished gradual learning rate warmup to 0.001.
500/500 [==============================] - 6s 11ms/step - loss: 0.0583 - accuracy: 0.9825 - lr: 0.0010
Epoch 4/24
500/500 [==============================] - 5s 11ms/step - loss: 0.0490 - accuracy: 0.9850 - lr: 0.0010
Epoch 5/24
500/500 [==============================] - 5s 10ms/step - loss: 0.0401 - accuracy: 0.9875 - lr: 0.0010
Epoch 6/24
500/500 [==============================] - 5s 11ms/step - loss: 0.0355 - accuracy: 0.9886 - lr: 0.0010
Epoch 7/24
500/500 [==============================] - 5s 10ms/step - loss: 0.0308 - accuracy: 0.9902 - lr: 0.0010
Epoch 8/24
500/500 [==============================] - 5s 10ms/step - l

In [21]:
result=mnist_model.predict(mnist_images[:5])

In [22]:
result.shape

(5, 10)

In [23]:
np.argmax(result[0,:])

5

In [1]:
import shutil
shutil.make_archive('/home/nmic/project/a','zip','/home/nmic/project/')

'/home/nmic/project/a.zip'

In [6]:
import glob
glob.glob('/home/nmic/project/*')

['/home/nmic/project/all_model.py',
 '/home/nmic/project/__pycache__',
 '/home/nmic/project/utils.py']

In [10]:
import os
os.getcwd()

'/home/nmic/project'